# TWO TOY EXAMPLES - CVXPY (ECOS)

**authors:** Jozef Hanč, Martina Hančová  
P. J. Šafárik University in Košice, Slovakia  
<jozef.hanc@upjs.sk>

 
### **Using Python CVXPY language, solver ECOS**

http://www.cvxpy.org  
**Ref.** DIAMOND, S. and BOYD, S., 2016. CVXPY: A Python-Embedded Modeling Language for Convex Optimization.  
In: Journal of Machine Learning Research, Vol. 17, No. 83, s. 1–5.  
AGRAWAL, A. et al (2018). A rewriting system for convex optimization problems. Journal of Control and Decision, 5(1), 42–60. 

**version of CVXPY:** 1.0.1  
**version of Python:** 2.7.14

##  1. High School Toy Example - Finding minimum of a simple convex  quadratic function

**Problem:** Find a minimum for the following quadratic function over $\mathbb{R}^+_0$:
>$f(x) = a x^2+b x+c, \\ a = 24, b = -227, c = 13934$

In [5]:
# Importing real number division
from __future__ import print_function
from __future__ import division

### Analytic solution

In [6]:
# Using the high school formula x=-b/2a for the parabola vertex 
a, b = 24, -227
xv = -b/(2*a)
x0 = c

# Results
if xv<0:
    print('Analytic solution =', x0)
else:
    print('Analytic solution =', xv)

Analytic solution = 4.72916666667


In [7]:
# Using SymPy - exact analytic solution
import sympy as sp

# Construct the problem
x = sp.var('x')
a, b, c = 24, -227, 12934
f = a*x**2 + b*x + c

# Solve the problem (via zero derivative)
dfdx = sp.diff(f,x)
xe = sp.solve(dfdx,x)[0]

# Results
print('SymPy exact analytic solution =', xe,'=', xe.n())
print('Optimal value =', f.subs(x,xe), '=', f.subs(x,xe).n())

SymPy exact analytic solution = 227/48 = 4.72916666666667
Optimal value = 1190135/96 = 12397.2395833333


### CVXPY solution

In [9]:
from cvxpy import *
import numpy as np

In [10]:
# Variables and constants
x = Variable()
a, b, c = 24, -227, 12934

# Construct the problem
objective = Minimize(a*x**2 + b*x + c)
constraints = [0 <= x]
prob = Problem(objective, constraints)

# Solve the problem (using default ECOS solver)
sol = prob.solve(verbose = True)

# Results
xcvx = x.value.tolist();
print('CVXPY solution =', xcvx)
print(prob.status, 'value =', prob.value)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+000  -4.513e+001  +2e+002  5e-001  4e-001  1e+000  8e+001    ---    ---    1  1  - |  -  - 
 1  -2.820e+002  -2.875e+002  +8e+000  1e-001  1e-001  5e+000  3e+000  0.9654  5e-004   2  2  2 |  0  0
 2  -5.161e+002  -5.084e+002  +8e-001  2e-002  4e-002  1e+001  3e-001  0.9193  1e-002   2  2  2 |  0  0
 3  -5.272e+002  -5.265e+002  +1e-001  3e-003  4e-003  1e+000  4e-002  0.8582  1e-002   2  2  2 |  0  0
 4  -5.369e+002  -5.369e+002  +7e-003  2e-004  2e-004  4e-002  3e-003  0.9486  1e-002   1  1  1 |  0  0
 5  -5.368e+002  -5.368e+002  +2e-004  5e-006  6e-006  1e-003  7e-005  0.9721  1e-004   1  1  1 |  0  0
 6  -5.368e+002  -5.368e+002  +2e-005  5e-007  6e-007  2e-004  7e-006  0.9123  1e-002   1  1  1 |  0  0
 7  -5.368e+002  -5.368e+002  +3e-007  7e-009  9e-009  6e-006  1e-007  0.9890  4e-003  

## Comparison
- **ERRORS**

In [12]:
# absolute errors
print("Error in the high school formula:", np.abs(xa - xe))
print("Error in CVXPY:", np.abs(xcvx-xe))

Error in the high school formula: 0
Error in CVXPY: 3.23957985495227e-5


## 2. Time Series Toy Example - Model fitting: Double Ordinary Least Squares (DOOLSE)

**Data**   
We are given an 24-hours observation $X$ of a time series $X(t):$ 

```
X = 39.2, 41.1, 40.0, 43.0, 38.8, 40.9, 37.7, 44.9,  
    44.8, 40.6, 46.4, 47.6, 47.7, 47.3, 45.2, 52.7,
    49.9, 43.8, 48.4, 43.4, 46.6, 43.5, 43.8, 41.9
```

**Modeling data - classical linear regression**  
We fit data by a classical linear regression model in the form:  

$X(t) = a + b\cos\left(\frac{2\pi}{24}\, t\right)+ c\sin\left(\frac{2\pi}{24}\, t\right)+\varepsilon(t), t = 1,2,\ldots,n$

where 

- $a,b,c \in \mathbb{R} $ and $n=24$
- $\varepsilon(t)$ is a noise with variance $\nu=\sigma^2>0$

**Estimator for variance $\boldsymbol{\nu}$** 

To estimate $\nu=\sigma^2$, we use *the DOuble Ordinary Least Squares Estimator (DOOLSE)*  
which is the result of the following convex optimization problem:

$ \text{minimize }\, f_0(\nu)=||\, \mathrm{S}(X)-\nu\mathrm{I}\, {||}^2, \nu \in \mathbb{R} $  
$\text{subject to }\, \nu > 0$

where

- $||\cdot||$ is the standard Euclidean matrix norm, $\mathrm{I}$ is the $n\times n$ identity matrix
- $ \mathrm{S}(X)$ is a matrix of the observation $X$ residuals determined by data $X$  
and the ordinary least squares estimation of the time series trend $a + b\cos\left(\frac{2\pi}{24}\, t\right)+ c\sin\left(\frac{2\pi}{24}\, t\right)$


**Note.** From the projection theory of vector (Hilbert) spaces, we know these important facts:

- $\mathrm{S}(X) = \mathrm{M_F}\, {XX}'\, \mathrm{M_F}$, where $\mathrm{M_F} = \mathrm{I} - \mathrm{F}(\mathrm{F'F})^{-1}\mathrm{F'}$ and $\mathrm{F}$ is a design model matrix
- The DOOLSE optimization problem for $\nu$ above has an analytical solution:  
$\hspace{2cm}\hat{\nu}= \dfrac{1}{n} \mathbf{tr} \,{\mathrm{S(X)}}$
- If $X$ has a normal distribution, this DOOLSE estimator is equal to the maximum likelihood estimator (MLE).

**Ref.** Štulajter, F. (2002). Predictions in Time Series Using Regression Models. New York: Springer.

## Analytical solution  

In [13]:
# Using numpy
import numpy as np
from math import sin,cos
from numpy.linalg import inv, norm
from __future__ import division
from __future__ import print_function

In [14]:
# Model parameters and design matrix F
n, om = 24, 2*np.pi/24

Fc = np.mat([[1]*n, [cos(om*t) for t in range(1,n+1)],
                    [sin(om*t) for t in range(1,n+1)]])

F = Fc.T; I = np.identity(n)
MF = I - F*inv(Fc*F)*Fc

# Problem data X and residual matrix S(X)
Xc = np.mat([39.2, 41.1, 40.0, 43.0, 38.8, 40.9, 37.7, 44.9, 
             44.8, 40.6, 46.4, 47.6, 47.7, 47.3, 45.2, 52.7,
             49.9, 43.8, 48.4, 43.4, 46.6, 43.5, 43.8, 41.9])
X = Xc.T
SX = MF*X*X.T*MF

# Solve the problem - analytic formula v = (1/n) tr S(X)
va = (1/n)*np.trace(SX)

In [15]:
# Results
print("Analytic DOOLSE =", va)

# Optimal value
print("Optimal value for DOOLSE =", norm(SX-va*I)**2)

Analytic DOOLSE = 4.738731006527846
Optimal value for DOOLSE = 12395.475496830104


In [17]:
# Using SymPy - exact analytic solution
import sympy as sp

In [18]:
# Model parameters and design matrix F
n, om = 24, 2*sp.pi/24

Fc = sp.Matrix([[1]*n, [sp.cos(om*t) for t in range(1,n+1)],
                       [sp.sin(om*t) for t in range(1,n+1)]])

F = Fc.T; I = sp.eye(n)
MF = (I - F*(Fc*F)**(-1)*Fc).expand()

# Problem data and residual matrix S(X)
Xfloat = sp.Matrix([39.2, 41.1, 40.0, 43.0, 38.8, 40.9, 37.7, 44.9, 
                     44.8, 40.6, 46.4, 47.6, 47.7, 47.3, 45.2, 52.7,
                     49.9, 43.8, 48.4, 43.4, 46.6, 43.5, 43.8, 41.9])

# Conversion data X to rationals
X = sp.Matrix([sp.sympify(str(Xfloat[i]), rational=True) for i in range(n)])
Xc = X.T
SX = (MF*X*Xc*MF).expand()

# Solve the problem - analytic formula, exact solution
ve = sp.Rational(1,n)*np.trace(SX)

In [19]:
# Results
ven = sp.N(ve,20)  # conversion to float with 20 decimals
print("Exact analytic DOOLSE =", ve,"=", ven)

Exact analytic DOOLSE = -1423*sqrt(3)/1152 - 24389*sqrt(6)/28800 - 40907*sqrt(2)/28800 + 63137/5760 = 4.7387310065278493224


## CVXPY solution

In [20]:
from cvxpy import *

In [21]:
# Model parameters and design matrix F
n, om = 24, 2*np.pi/24

Fc = np.mat([[1]*n, [cos(om*t) for t in range(1,n+1)],
                    [sin(om*t) for t in range(1,n+1)]])

F = Fc.T; I = np.identity(n)
MF = I - F*inv(Fc*F)*Fc

# Problem data and residual matrix S(X)
Xc = np.mat([39.2, 41.1, 40.0, 43.0, 38.8, 40.9, 37.7, 44.9, 
             44.8, 40.6, 46.4, 47.6, 47.7, 47.3, 45.2, 52.7,
             49.9, 43.8, 48.4, 43.4, 46.6, 43.5, 43.8, 41.9])
X = Xc.T
SX = MF*X*Xc*MF

# Variables for the objective in DOOLSE
v = Variable()

# Construct the problem for DOOLSE
objective = Minimize(sum_squares(SX-v*I))
constraints = [0 <= v]
prob = Problem(objective, constraints)

# Solution for DOOLSE
prob.solve(verbose = True)

vcvx = v.value.tolist();

print('DOOLSE =', vcvx)
print(prob.status, 'value for DOOLSE =', prob.value)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+000  -1.037e+000  +2e+002  6e-001  4e-002  1e+000  4e+001    ---    ---    1  1  - |  -  - 
 1  +4.016e+002  +4.905e+002  +1e+002  2e+000  2e-001  1e+002  3e+001  0.5991  6e-001   1  1  1 |  0  0
 2  +4.296e+002  +4.417e+002  +7e+000  2e-001  1e-002  1e+001  2e+000  0.9382  3e-003   2  1  1 |  0  0
 3  +7.079e+002  +8.135e+002  +2e+000  4e-001  1e-002  1e+002  6e-001  0.9890  3e-001   2  1  1 |  0  0
 4  +1.062e+003  +1.119e+003  +3e-001  1e-001  3e-003  6e+001  9e-002  0.8423  8e-003   2  1  1 |  0  0
 5  +1.749e+003  +2.546e+003  +1e-001  5e-001  1e-002  8e+002  4e-002  0.9406  4e-001   3  2  2 |  0  0
 6  +1.438e+003  +1.556e+003  +6e-002  1e-001  2e-003  1e+002  2e-002  0.8597  3e-001   3  2  1 |  0  0
 7  +2.032e+003  +2.682e+003  +1e-002  1e-001  2e-003  7e+002  3e-003  0.8506  2e-002  

## Comparison
- **ERRORS**

In [23]:
# absolute errors
print("error in analytic DOOLSE:", np.abs(va - ven).n())
print("error in CVXPY DOOLSE:", np.abs(vcvx - ven).n())

error in analytic DOOLSE: 3.03232374106177e-15
error in CVXPY DOOLSE: 3.98826223969889e-7
